<a href="https://colab.research.google.com/github/TravellerOnCode/Working-with-TFLite-and-Flutter/blob/master/Grading_TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Libraries

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [ ]:
from tqdm import tqdm
import numpy as np


In [ ]:
#Required Libraries
import matplotlib.pyplot as plt 
import fnmatch 
from tqdm import tqdm
import pandas as pd
import io
import math
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import cohen_kappa_score

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import unicodedata


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'ag

#Loading and Cleaning Data

In [ ]:
#Load the data and store in a data frame
def load_data():
    
    df = pd.read_csv(r'essays_and_scores.csv',encoding='latin-1')
    #data = df[['essay_id','essay_set','essay','rater1_domain1','rater1_domain1','domain1_score']].copy()
    data = df[['essay_set','essay','rater1_domain1','rater1_domain1','domain1_score']].copy()
    data = data.dropna() #drop all NaN values
    #print(data)
    #check details of the data
    print(data['essay'].apply(len).describe())
    return data

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

collection = contraction_mapping.keys()

def expand_words(s):
    if s in collection:
        return contraction_mapping[s]
    else:
        return s
    

In [ ]:
#garbage = {'@ORGANIZATION?':'organization','@NAME?':'number','@NUM':'number','@CAPS?':'name','@DR':'name','Dr?':'doctor','Mr?':'mister','Mrs?':'mistress','@LOCATION?':'location','@CITY?':'location','@PERSON?':'name','@MONEY?':'money','@DATE?':'date','@MONTH?':'month'}
garbage = {'ORGANIZATION?':'organization','NAME?':'name','NUM':'number','CAPS?':'name','DR':'name','Dr?':'doctor','Mr?':'mister','Mrs?':'mistress',
           'LOCATION?':'location','CITY?':'location','PERSON?':'name','MONEY?':'money','DATE?':'date','MONTH?':'month'}

def replace_garbage_words(s):
    for i in garbage:
        match = fnmatch.fnmatch(s, i)
        if match == True:
              return (garbage[i])
    return s

In [ ]:
def clean_data(data):
    essay_data = []
    essay_lengths = []
    essay_scores = []
    essay_category = []
    max_score = [12,6,3,3,4,4,24,60]
    for index, row in data.iterrows():
    #for text in data['essay']:
        text = row['essay']
        length = len(text.split(' '))
        #if length<50:
         #     continue
        #if length>2000:
         #     continue
        #Normalise (normalize) unicode data in Python to remove umlauts, accents etc.
        unicode_data = text
        text = unicodedata.normalize('NFKD', unicode_data).encode('ASCII', 'ignore')
        text = str(text)
        #print (normal)

        # split into words
        tokens = word_tokenize(text)
        #replace garbage words
        tokens = [replace_garbage_words(w) for w in tokens]  
        tokens = [expand_words(w) for w in tokens]
    
        #for n,w in enumerate(tokens):
        #  tokens[n]=replace_garbage_words(w)
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove remaining tokens that are not alphabetic
        words = [word for word in stripped if word.isalpha()]

        # convert to lower case
        words = [w.lower() for w in words]
    

        str1 = ""
        for i in words:
            str1 = str1 + i + " "
        str2 = str1[1:]
        str2.lstrip()
        essay_data.append(str2)
        essay_lengths.append(length)
    
        category = row['essay_set']
        score = row['domain1_score']
        score = round((score/max_score[category-1])*10) #taking percentile scores
        essay_scores.append(score)
        essay_category.append(category)

    new_data = pd.DataFrame(list(zip(essay_data,essay_scores)),columns = ['Essay','Score'])
    print(new_data['Essay'].apply(len).describe())
    return (new_data,essay_lengths,essay_scores,essay_category)

In [ ]:
def generate_test_train(new_data):
    #shuffle the dataset before splitting
    new_data1 = shuffle(new_data)
    #new_data1 = shuffle(new_data1)
    #new_data1 = shuffle(new_data1)

    train = new_data1.sample(frac=0.8)
    new_data1.drop(train.index, axis=0, inplace=True)
    valid = new_data1.sample(frac=0.5)
    new_data1.drop(valid.index, axis=0, inplace=True)
    test = new_data1
    print(train.shape , test.shape , valid.shape )

    def return_data(df):
        return list(df['Essay']), np.array(df['Score'])

    # Apply it to the three splits
    train_essay, train_score = return_data(train)
    valid_essay, valid_score = return_data(valid)
    test_essay, test_score = return_data(test)
    #print(train_essay[0], train_score[0])
    #return a tuple of values 
    return (train_essay,train_score,valid_essay,valid_score,test_essay,test_score)

#Calling Functions

In [ ]:
data = load_data()
print('*******************')

count    12977.000000
mean      1215.871234
std        958.321377
min          8.000000
25%        527.000000
50%        900.000000
75%       1670.000000
max       6098.000000
Name: essay, dtype: float64
*******************


In [ ]:
new_data,essay_lengths,essay_scores,essay_category = clean_data(data)

count    12977.000000
mean      1171.974262
std        915.155314
min          9.000000
25%        513.000000
50%        869.000000
75%       1613.000000
max       5963.000000
Name: Essay, dtype: float64


In [ ]:
new_data

,Essay,Score
0,dear local newspaper i think effects computer...,7
1,dear name name i believe that using computers...,8
2,dear name name name more and more people use ...,6
3,dear local newspaper name i have found that ma...,8
4,dear location i know having computers has a p...,7
...,...,...
12972,in most stories mothers and daughters are eit...,6
12973,i never understood the meaning laughter is th...,5
12974,when you laugh is name out of habit or is name...,7
12975,trippin on fences i am years young and in tho...,7


In [ ]:
train_essay,train_score,valid_essay,valid_score,test_essay,test_score = generate_test_train(new_data)
print(train_essay[10])
print(train_score[10])

(10382, 2) (1297, 2) (1298, 2)
in the excerpt the name mast by name the architects of the empire state building faced obstacles in attempting to allow dirigibles to dock thereone of the problems the builders encountered was that the dirigible didnt have a suitable landing area the architechs concluded that they couldnt just drop a name mast on top of their building because it would add stress to its frame so they had to strengthen and modify it to make it workanother obstacle was the fact that most dirigibles ran on hydrogen which is highly flamable the builders realized how dangerous that could be especially above a densely populated area such as downtown new york they also realized that nature itself was their biggest obstace winds were constatly shifting due to violent air currents and they knew that this idea was neither practical nor safe lastly there was a law against airships flying too low over urban areas which would make their idea illegal and unsuccessful 
8


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#Prepare the Data

In [ ]:
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [ ]:
# using keras tokenizer here
token = tf.keras.preprocessing.text.Tokenizer(oov_token = "<OOV>")
    
#token = text.Tokenizer(num_words=None,oov_token=oov_tok)
#max_len = 1100
max_len = 1024
token.fit_on_texts(list(train_essay) + list(valid_essay))
word_index = token.word_index
len(word_index)

40027

In [ ]:
type(token)

keras_preprocessing.text.Tokenizer

In [ ]:
type(word_index)

dict

In [ ]:
import json
with open('tokenizer.txt','w') as file:
    file.write(json.dumps(word_index))

In [ ]:
with open('tokenizer.txt','r') as f, open('tokenizer_labels.txt', 'w') as file:
    for line in f:
       for word in line.split(','):
           file.write(word + '\n')

In [ ]:
xtrain_seq = token.texts_to_sequences(train_essay)
xvalid_seq = token.texts_to_sequences(valid_essay)
xtest_seq = token.texts_to_sequences(test_essay)

#zero pad the sequences
preproc_train = tf.keras.preprocessing.sequence.pad_sequences(xtrain_seq, maxlen=max_len,padding=padding_type, truncating=trunc_type)
preproc_valid = tf.keras.preprocessing.sequence.pad_sequences(xvalid_seq, maxlen=max_len,padding=padding_type, truncating=trunc_type)
preproc_test = tf.keras.preprocessing.sequence.pad_sequences(xtest_seq, maxlen=max_len,padding=padding_type, truncating=trunc_type)


In [ ]:
preproc_train = tf.cast(preproc_train,tf.float32)
preproc_valid = tf.cast(preproc_valid,tf.float32)
preproc_test = tf.cast(preproc_test,tf.float32)

train_score = tf.cast(train_score,tf.float32)
test_score = tf.cast(test_score,tf.float32)
valid_score = tf.cast(valid_score,tf.float32)

In [ ]:
type(preproc_test[1])

tensorflow.python.framework.ops.EagerTensor

In [ ]:
len(test_score)

1297

#GloVe Embeddings

In [ ]:
!gsutil cp gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt glove.6B.200d.txt

Copying gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt...
| [1 files][661.3 MiB/661.3 MiB]                                                
Operation completed over 1 objects/661.3 MiB.                                    


In [ ]:
embeddings_index = {}
f = open('glove.6B.200d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [00:24, 16566.43it/s]

Found 400000 word vectors.


In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 200))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 40027/40027 [00:00<00:00, 593656.29it/s]


#Building the Model

In [ ]:
def build_model():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Embedding(len(word_index) + 1,
                     200,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='relu'))
    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer = tf.keras.optimizers.RMSprop(
                            learning_rate=0.001),
        metrics=['mae'])
    print(model.summary())
    return model

In [ ]:
model = build_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1024, 200)         8005600   
_________________________________________________________________
bidirectional (Bidirectional (None, 1024, 512)         935936    
_________________________________________________________________
dropout (Dropout)            (None, 1024, 512)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               295424    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 9,237,089
Trainable params: 1,231,489
Non-trainable params: 8,005,600
______________________________________

#Train the Model

In [ ]:
H = model.fit(preproc_train,
         train_score,
         validation_data=(preproc_valid, valid_score),
         batch_size=64,
         epochs=20,
         verbose=1)

Epoch 1/20
163/163 [==============================] - 54s 332ms/step - loss: 6.8003 - mae: 2.0926 - val_loss: 5.6934 - val_mae: 1.8462
Epoch 2/20
163/163 [==============================] - 54s 331ms/step - loss: 5.3346 - mae: 1.8384 - val_loss: 3.4871 - val_mae: 1.4858
Epoch 3/20
163/163 [==============================] - 54s 329ms/step - loss: 3.4114 - mae: 1.4444 - val_loss: 2.7634 - val_mae: 1.2887
Epoch 4/20
163/163 [==============================] - 53s 328ms/step - loss: 3.1057 - mae: 1.3797 - val_loss: 2.7035 - val_mae: 1.2966
Epoch 5/20
163/163 [==============================] - 54s 329ms/step - loss: 2.9471 - mae: 1.3416 - val_loss: 2.4537 - val_mae: 1.2183
Epoch 6/20
163/163 [==============================] - 54s 329ms/step - loss: 2.9434 - mae: 1.3372 - val_loss: 2.4788 - val_mae: 1.2182
Epoch 7/20
163/163 [==============================] - 54s 328ms/step - loss: 2.7633 - mae: 1.2943 - val_loss: 2.6706 - val_mae: 1.2357
Epoch 8/20
163/163 [==============================] - 5

In [ ]:
H = model.fit(preproc_train,
         train_score,
         validation_data=(preproc_valid, valid_score),
         batch_size=64,
         epochs=20,
         verbose=1)

Epoch 1/20
163/163 [==============================] - 77s 470ms/step - loss: 6.6682 - mae: 2.0545 - val_loss: 4.7421 - val_mae: 1.7662
Epoch 2/20
163/163 [==============================] - 76s 464ms/step - loss: 4.1149 - mae: 1.5832 - val_loss: 3.3812 - val_mae: 1.5020
Epoch 3/20
163/163 [==============================] - 76s 463ms/step - loss: 3.9500 - mae: 1.5491 - val_loss: 2.9670 - val_mae: 1.3230
Epoch 4/20
163/163 [==============================] - 76s 464ms/step - loss: 3.1260 - mae: 1.3823 - val_loss: 2.9272 - val_mae: 1.3815
Epoch 5/20
163/163 [==============================] - 76s 464ms/step - loss: 2.9979 - mae: 1.3499 - val_loss: 2.5317 - val_mae: 1.2550
Epoch 6/20
163/163 [==============================] - 75s 463ms/step - loss: 2.7428 - mae: 1.2896 - val_loss: 2.5406 - val_mae: 1.2193
Epoch 7/20
163/163 [==============================] - 76s 463ms/step - loss: 2.6439 - mae: 1.2645 - val_loss: 3.8080 - val_mae: 1.5483
Epoch 8/20
163/163 [==============================] - 7

In [ ]:
import matplotlib.pyplot as plt
history_dict = H
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

#Evaluating the Model

In [ ]:
y_pred = model.predict(preproc_test)

In [ ]:
def Kappa_Score(test_score,y_pred):
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(test_score,np.around(y_pred),weights='quadratic')
    print("Kappa Score: {}".format(result))
    return result

In [ ]:
r = Kappa_Score(test_score,y_pred)

Kappa Score: 0.7640648375830119


In [ ]:
#new
r = Kappa_Score(test_score,y_pred)

Kappa Score: 0.7505809274317109


Reduced Inputs

In [ ]:
preproc_test_reduced = preproc_test[:50]
test_score_reduced = test_score[:50]

In [ ]:
y_pred_reduced = model.predict(preproc_test_reduced)

In [ ]:
r = Kappa_Score(test_score_reduced,y_pred_reduced)

Kappa Score: 0.7021667865995043


#Saving and Converting the Model to TFLite

In [ ]:
export_dir = 'saved_model/1'
tf.saved_model.save(model,export_dir)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/1/assets


In [ ]:

# Show model size in KBs.
#original_model = 
float_model_size = len(model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Convert to TFLite (without optimization) 

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

In [ ]:
tflite_model = converter.convert()
print('Converted !')

Converted !


In [ ]:
tflite_model_file = 'model_grading_.tflite'

with open(tflite_model_file, "wb") as f:
      f.write(tflite_model)

In [ ]:
# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Float model size = 37154KBs.


Convert to TFLite (with optimization) 

In [ ]:
#New Approach
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [optimization]
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
#converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
#mnist_train, _ = tf.keras.datasets.mnist.load_data()
texts = tf.cast(preproc_train[0], tf.float32)
data_ds = tf.data.Dataset.from_tensor_slices((texts)).batch(1)
def representative_data_gen():
  for input_value in data_ds.take(100):
    yield [input_value]

converter.representative_dataset = representative_data_gen

In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

In [ ]:
tflite_model = converter.convert()
print('Converted !')

ValueError: ignored

In [ ]:
tflite_model_file = 'model_grading_2.tflite'

with open(tflite_model_file, "wb") as f:
      f.write(tflite_model)

In [ ]:

# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Float model size = 37149KBs.


In [ ]:

# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Float model size = 37090KBs.


#Testing with an Interpreter

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
#output = interpreter.tensor(interpreter.get_output_details()[0]["index"])


In [ ]:
# Gather results for the randomly sampled test images
predictions2 = []
test_scores2 = []
test_datas2 = []

for i in range(50):
  text = preproc_test[i]
  score = test_score[i]
  #text = np.expand_dims(text, axis=0).astype(np.float32)
  text = tf.expand_dims(text,0)
  interpreter.set_tensor(input_index, text)
  interpreter.invoke()
  #print(output[0])
  predictions2.append(interpreter.get_tensor(output_index))
  #predictions.append(output[0])
  test_scores2.append(score)
  test_datas2.append(text)
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
y_pred

array([[8.16938  ],
       [9.272753 ],
       [4.4491844],
       ...,
       [8.657941 ],
       [9.52352  ],
       [5.1031284]], dtype=float32)

In [ ]:
p = []
for i in predictions2:
  l = []
  l.append(i[0][0])
  p.append(l)
p = np.array(p)
print(p)

[[8.169376 ]
 [9.272752 ]
 [4.4491835]
 [7.1341734]
 [6.3053713]
 [4.063136 ]
 [9.403208 ]
 [9.558133 ]
 [7.890703 ]
 [6.9883833]
 [8.98264  ]
 [4.2297544]
 [6.8236227]
 [5.155761 ]
 [3.2727332]
 [7.1645994]
 [6.5297613]
 [5.9954357]
 [5.1021347]
 [6.1679497]
 [4.2695417]
 [8.333868 ]
 [9.586123 ]
 [3.203994 ]
 [4.856967 ]
 [6.689765 ]
 [9.369832 ]
 [6.3990326]
 [6.6721754]
 [8.477364 ]
 [6.432741 ]
 [7.180423 ]
 [7.1647053]
 [3.90539  ]
 [6.918458 ]
 [9.423333 ]
 [5.5150948]
 [8.722359 ]
 [8.405177 ]
 [5.991638 ]
 [9.04976  ]
 [7.4759064]
 [2.5397172]
 [7.0149364]
 [3.0809972]
 [6.4736013]
 [8.369404 ]
 [2.869701 ]
 [6.2348747]
 [5.373827 ]]


In [ ]:
print(test_scores)

[<tf.Tensor: shape=(), dtype=float32, numpy=9.0>, <tf.Tensor: shape=(), dtype=float32, numpy=8.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=8.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>, <tf.Tensor: shape=(), dtype=float32, numpy=8.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=7.0>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=

In [ ]:
print(len(test_scores2))
print(len(predictions))
print(len(test_score))
#r2 = Kappa_Score(test_scores2,p)

50
50
1297


In [ ]:
r2 = Kappa_Score(test_score[:50],p)

Kappa Score: 0.8758240765826786


In [ ]:
print("Comparison between two Models !")
print("Original Model Score: ",r)
print("TFLite Model: Optimization = Storage ",r2)

Comparison between two Models !
Original Model Score:  0.7640648375830119
TFLite Model: Optimization = Storage  0.8758240765826786


In [ ]:
print("TFLite Model: Optimization = Speed ",r3)

#Quantization of the Model

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
#converter.optimizations = [optimization]


In [ ]:
# Re-convert the model to TF Lite using quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

In [ ]:
# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

#Using a different Model

In [ ]:
from tensorflow.keras.layers import Embedding


In [ ]:
num_tokens = len(word_index) + 1
embedding_dim = 200

In [ ]:
embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    input_length=max_len,
    trainable=False,
)

In [ ]:
model.add(tf.keras.layers.Embedding(len(word_index) + 1,
                     200,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    

In [ ]:
model2 = tf.keras.Sequential()

model2.add(embedding_layer)
model2.add(tf.keras.layers.Conv1D(128, 5, activation="relu"))
model2.add(tf.keras.layers.MaxPooling1D(5))
model2.add(tf.keras.layers.Conv1D(128, 5, activation="relu"))   
model2.add(tf.keras.layers.MaxPooling1D(5))
model2.add(tf.keras.layers.Conv1D(128, 5, activation="relu"))
model2.add(tf.keras.layers.GlobalMaxPooling1D())
model2.add(tf.keras.layers.Dense(128, activation="relu"))
model2.add(tf.keras.layers.Dropout(0.5))
model2.add(tf.keras.layers.Dense(1, activation="relu")) 
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 200)         8005600   
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 1020, 128)         128128    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 204, 128)          0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 200, 128)          82048     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 40, 128)           0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 36, 128)           82048     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)              

In [ ]:
model2.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer = tf.keras.optimizers.RMSprop(
                            learning_rate=0.001),
        metrics=['mae'])

In [ ]:
H = model2.fit(preproc_train,
         train_score,
         validation_data=(preproc_valid, valid_score),
         batch_size=64,
         epochs=20,
         verbose=1)

Epoch 1/20
163/163 [==============================] - 5s 29ms/step - loss: 7.3085 - mae: 2.0107 - val_loss: 3.9575 - val_mae: 1.5464
Epoch 2/20
163/163 [==============================] - 5s 28ms/step - loss: 4.6514 - mae: 1.7179 - val_loss: 2.4990 - val_mae: 1.2107
Epoch 3/20
163/163 [==============================] - 5s 28ms/step - loss: 3.9906 - mae: 1.5816 - val_loss: 2.5574 - val_mae: 1.2491
Epoch 4/20
163/163 [==============================] - 5s 28ms/step - loss: 3.6889 - mae: 1.5152 - val_loss: 2.4632 - val_mae: 1.2298
Epoch 5/20
163/163 [==============================] - 5s 28ms/step - loss: 3.3275 - mae: 1.4428 - val_loss: 2.4102 - val_mae: 1.2101
Epoch 6/20
163/163 [==============================] - 5s 28ms/step - loss: 3.0290 - mae: 1.3703 - val_loss: 2.3481 - val_mae: 1.1829
Epoch 7/20
163/163 [==============================] - 5s 28ms/step - loss: 2.7547 - mae: 1.3114 - val_loss: 2.8208 - val_mae: 1.3433
Epoch 8/20
163/163 [==============================] - 5s 28ms/step - 

Test the Model

In [ ]:
y_pred2 = model2.predict(preproc_test)


In [ ]:
r = Kappa_Score(test_score,y_pred2)


Kappa Score: 0.7176941936328558


In [ ]:
preproc_test_reduced2 = preproc_test[:50]
test_score_reduced2 = test_score[:50]


In [ ]:
y_pred_reduced2 = model2.predict(preproc_test_reduced2)
r = Kappa_Score(test_score_reduced2,y_pred_reduced2)

Kappa Score: 0.670495198644323


In [ ]:
export_dir = 'saved_model/2'
tf.saved_model.save(model,export_dir)

INFO:tensorflow:Assets written to: saved_model/2/assets


INFO:tensorflow:Assets written to: saved_model/2/assets


TFLite

In [ ]:
texts = tf.cast(preproc_train[0], tf.float32)
data_ds = tf.data.Dataset.from_tensor_slices((texts)).batch(2)
def representative_data_gen():
  for input_value in data_ds.take(100):
    yield [input_value]

converter.representative_dataset = representative_data_gen


In [ ]:
#New Approach
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
#converter.optimizations = [optimization]
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#mnist_train, _ = tf.keras.datasets.mnist.load_data()

In [ ]:

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()
print('Converted !')

Converted !


In [ ]:
tflite_model_file = 'model_grading_quant2.tflite'

with open(tflite_model_file, "wb") as f:
      f.write(tflite_model)

In [ ]:
# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Float model size = 8127KBs.


In [ ]:
# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

Float model size = 8127KBs.


In [ ]:
# Show model size in KBs.
float_model_size = len(tflite_model) / 1024
print('Float model size (float16) = %dKBs.' % float_model_size)

Float model size (float16) = 16246KBs.


#Testing new interpreter

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
#output = interpreter.tensor(interpreter.get_output_details()[0]["index"])


In [ ]:
# Gather results for the randomly sampled test images
predictions2 = []
test_scores2 = []
test_datas2 = []

for i in range(50):
  text = preproc_test[i]
  score = test_score[i]
  #text = np.expand_dims(text, axis=0).astype(np.float32)
  text = tf.expand_dims(text,0)
  interpreter.set_tensor(input_index, text)
  interpreter.invoke()
  #print(output[0])
  predictions2.append(interpreter.get_tensor(output_index))
  #predictions.append(output[0])
  test_scores2.append(score)
  test_datas2.append(text)
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
p = []
for i in predictions2:
  l = []
  l.append(i[0][0])
  p.append(l)
p = np.array(p)
print(p)

[[4.6155157]
 [3.954429 ]
 [7.147621 ]
 [4.7092924]
 [5.7188396]
 [7.6974525]
 [4.0079374]
 [6.6991386]
 [7.392479 ]
 [5.359309 ]
 [1.6638725]
 [9.032124 ]
 [6.8230877]
 [6.624441 ]
 [4.371544 ]
 [9.78329  ]
 [6.5079107]
 [7.2174625]
 [6.522917 ]
 [8.107827 ]
 [5.6272507]
 [4.213863 ]
 [6.212867 ]
 [5.7143955]
 [6.1007137]
 [6.5669994]
 [5.8562155]
 [4.5475397]
 [9.486784 ]
 [6.542404 ]
 [4.5877132]
 [6.058076 ]
 [7.505533 ]
 [6.6254854]
 [6.1992455]
 [6.3393593]
 [6.481964 ]
 [7.1225805]
 [8.634988 ]
 [8.7665615]
 [7.043956 ]
 [4.7625523]
 [7.662154 ]
 [5.204397 ]
 [2.9797738]
 [7.8173246]
 [6.9222193]
 [7.417098 ]
 [7.5704737]
 [4.240813 ]]


In [ ]:
print(len(test_scores2))
#print(len(predictions))
print(len(test_score))
#r2 = Kappa_Score(test_scores2,p)

In [ ]:
r2 = Kappa_Score(test_score[:50],p)

Kappa Score: 0.6666036129764494
